In [0]:
from pyspark.sql.functions import avg, stddev, weekofyear ,col
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, TimestampType
from datetime import datetime

In [0]:
#Зчитування silver data
gold_df = spark.read.format("delta").load("/FileStore/delta/silver_stock")

In [0]:

# Створення таблиці з Добовими метриками
gold_daily = gold_df.groupBy("Year", "Month", "Day", "Company").agg(
    avg("Close").alias("avg_daily_close"),
    stddev("Close").alias("volatility_daily")
)

#  Створення таблиці з Тижневими метриками
gold_weekly = gold_df.withColumn("Week", weekofyear(col("Date"))) \
    .groupBy("Year", "Week", "Company").agg(
        avg("Close").alias("avg_weekly_close"),
        stddev("Close").alias("volatility_weekly")
    )

# Створення таблиці з Місячними метриками
gold_monthly = gold_df.groupBy("Year", "Month", "Company").agg(
    avg("Close").alias("avg_monthly_close"),
    stddev("Close").alias("volatility_monthly")
)

In [0]:
# Запис агрегованих таблиць у форматі delta
gold_daily.write.format("delta").mode("overwrite").option("mergeSchema", "true").save("/FileStore/delta/gold_daily_stock")
gold_weekly.write.format("delta").mode("overwrite").option("mergeSchema", "true").save("/FileStore/delta/gold_weekly_stock")
gold_monthly.write.format("delta").mode("overwrite").option("mergeSchema", "true").save("/FileStore/delta/gold_monthly_stock")

In [0]:
# Додавання даних до лог-таблиці
log_data = [(
    datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    "gold+_stock_data",
    "/FileStore/delta/silver_stock",
    None,
    None,
    None,
    gold_daily.count(),
    gold_weekly.count(),
    gold_monthly.count()
)]

log_gold_df = spark.createDataFrame(log_data, schema=log_schema)
#Запис даних до лог-таблиці "etl_log" у форматі delta
log_gold_df.write.format("delta").mode("append").save("/FileStore/delta/etl_log")
